# Azure AI Search backup and restore sample

This notebook demonstrates how to backup and restore a search index and migrate it to another instance.

The only pre-requsitite is that your search index has a `key` field that is `filterable` and `sortable`. If you don't have one, you can create a new field and assign unique values to your search index. 

It is important to note that only fields marked as `retrievable` can be successfully backed up and restored. It's crucial to consider whether or not you want your vector fields to be marked as `retrievable` in your search index. Marking vector fields as `retrievable` will allow you to backup and restore them and use them for any purpose, whereas NOT marking them as `retrievable` will save you storage costs, but the tradeoff is that you will not be able to backup and restore those fields.

Please review this sample and follow the instructions provided in this Jupyter Python notebook to backup and restore your Azure AI Search indexes.

In [1]:
! pip install azure-search-documents --pre
! pip install tqdm

  Using cached azure_core-1.29.6-py3-none-any.whl.metadata (36 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl (14 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached anyio-4.2.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl.metadata (34 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
   ---------------------------------------- 0.0/283.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/283.8 kB ? eta -:--:--
   ----- --------------------------------- 41.0/283.8 kB 487.6 kB/

This script demonstrates backing up and restoring an Azure AI Search index between two services. The `backup_and_restore_index` function retrieves the source index definition, creates a new target index, backs up all documents, and restores them to the target index.

In [4]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [5]:
import dotenv

dotenv.load_dotenv()

True

In [6]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient
import tqdm  
import os

def create_clients(endpoint, key, index_name):  
    search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(key))  
    index_client = SearchIndexClient(endpoint=endpoint, credential=AzureKeyCredential(key))  
    return search_client, index_client

def total_count(search_client):
    response = search_client.search(include_total_count=True, search_text="*", top=0)
    return response.get_count()
  
def search_results_with_filter(search_client, key_field_name):
    last_item = None
    response = search_client.search(search_text="*", top=100000, order_by=key_field_name).by_page()
    while True:
        for page in response:
            page = list(page)
            if len(page) > 0:
                last_item = page[-1]
                yield page
            else:
                last_item = None
        
        if last_item:
            response = search_client.search(search_text="*", top=100000, order_by=key_field_name, filter=f"{key_field_name} gt '{last_item[key_field_name]}'").by_page()
        else:
            break

def search_results_without_filter(search_client):
    response = search_client.search(search_text="*", top=100000).by_page()
    for page in response:
        page = list(page)
        yield page

def backup_and_restore_index(source_endpoint, source_key, source_index_name, target_endpoint, target_key, target_index_name):  
    # Create search and index clients  
    source_search_client, source_index_client = create_clients(source_endpoint, source_key, source_index_name)  
    target_search_client, target_index_client = create_clients(target_endpoint, target_key, target_index_name)  
  
    # Get the source index definition  
    source_index = source_index_client.get_index(name=source_index_name)
    non_retrievable_fields = []
    for field in source_index.fields:
        if field.hidden == True:
            non_retrievable_fields.append(field)
        if field.key == True:
            key_field = field

    if not key_field:
        raise Exception("Key Field Not Found")
    
    if len(non_retrievable_fields) > 0:
        print(f"WARNING: The following fields are not marked as retrievable and cannot be backed up and restored: {', '.join(f.name for f in non_retrievable_fields)}")
  
    # Create target index with the same definition 
    source_index.name = target_index_name
    target_index_client.create_or_update_index(source_index)
  
    document_count = total_count(source_search_client)
    can_use_filter = key_field.sortable and key_field.filterable
    if not can_use_filter:
        print("WARNING: The key field is not filterable or not sortable. A maximum of 100,000 records can be backed up and restored.")
    # Backup and restore documents  
    all_documents = search_results_with_filter(source_search_client, key_field.name) if can_use_filter else search_results_without_filter(source_search_client)

    print("Backing up and restoring documents:")  
    failed_documents = 0  
    failed_keys = []  
    with tqdm.tqdm(total=document_count) as progress_bar:  
        for page in all_documents:
            result = target_search_client.upload_documents(documents=page)  
            progress_bar.update(len(result))  
  
            for item in result:  
                if item.succeeded is not True:  
                    failed_documents += 1
                    failed_keys.append(page[result.index_of(item)].id)  
                    print(f"Document upload error: {item.error.message}")  
  
    if failed_documents > 0:  
        print(f"Failed documents: {failed_documents}")  
        print(f"Failed document keys: {failed_keys}")  
    else:  
        print("All documents uploaded successfully.")  
  
    print(f"Successfully backed up '{source_index_name}' and restored to '{target_index_name}'")  
    return source_search_client, target_search_client, all_documents  
  
# Replace with your service endpoints, keys, and index names


source_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
source_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
source_index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
target_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
target_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
target_index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") + "-new"

source_search_client, target_search_client, all_documents = backup_and_restore_index(source_endpoint, source_key, source_index_name, target_endpoint, target_key, target_index_name)  


Backing up and restoring documents:


100%|██████████| 123/123 [00:05<00:00, 21.62it/s]

All documents uploaded successfully.
Successfully backed up 'gptkbindex2' and restored to 'gptkbindex2-new'


The verify_counts function compares document counts between source and target indexes after backup and restore. It prints a message indicating if the document counts match or not.

In [7]:
def verify_counts(source_search_client, target_search_client):  
    source_document_count = source_search_client.get_document_count()  
    target_document_count = target_search_client.get_document_count()  
  
    print(f"Source document count: {source_document_count}")  
    print(f"Target document count: {target_document_count}")  
  
    if source_document_count == target_document_count:  
        print("Document counts match.")  
    else:  
        print("Document counts do not match.")  
  
# Call the verify_counts function with the search_clients returned by the backup_and_restore_index function  
verify_counts(source_search_client, target_search_client)  


Source document count: 123
Target document count: 123
Document counts match.
